In [1]:
import pandas as pd
import glob
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
import numpy as np
# https://scikit-learn.org/stable/modules/ensemble.html#extremely-randomized-trees

### Loading  Data

In [2]:
train_data = pd.read_csv("C:\\Kaggle competetion datasets\\Elo Merchent Category Recommendation\\Elo Feb 18th\\summary_card\\df_train_summary.csv")
test_data = pd.read_csv("C:\\Kaggle competetion datasets\\Elo Merchent Category Recommendation\\Elo Feb 18th\\summary_card\\df_test_summary.csv")

In [3]:
print(train_data.shape)
test_data.shape

(201917, 95)


(123623, 93)

### Functions to concatenate features, kfold splits

In [4]:
def feature_concat(train_data1,filepath):
    for file in filepath:
        data_features = pd.read_csv(file)
        train_data1=pd.merge(train_data1,data_features,how='left',on='card_id')
    return(train_data1)

In [5]:
def kfold_split(splitcount,train_data1,ignore_cols):  
    output_feature=['target']
    input_features=[x for x in train_data1.columns if x not in ignore_cols]
    train_X = train_data1[input_features]
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    train_X = train_X.select_dtypes(include=numerics)
    train_y = train_data1[output_feature]
    kf = StratifiedKFold(n_splits=splitcount, random_state=2018, shuffle=True)
    counter=0
    models=[]
    splits={}
    for dev_index, val_index in kf.split(train_X,train_data1['target_bin']):
        dev_X, val_X = train_X.loc[dev_index, :], train_X.loc[val_index, :]
        dev_y, val_y = train_y.loc[dev_index], train_y.loc[val_index]
        splits[counter] = [dev_X, val_X,dev_y, val_y]
        counter=counter+1
    return splits

### Concatenate features

In [6]:
%%time
file_list = glob.glob("C:/Kaggle competetion datasets/Elo Merchent Category Recommendation/Elo Feb 18th/features_feb22nd/*.csv")
train_features = feature_concat(train_data,file_list)

Wall time: 2min 15s


In [7]:
train_features.shape

(201917, 451)

In [8]:
# train_features=train_data.copy()

In [9]:
# # Create correlation matrix
# corr_matrix = train_features.corr().abs()

# # Select upper triangle of correlation matrix
# upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# # Find index of feature columns with correlation greater than 0.95
# to_drop = [column for column in upper.columns if any(upper[column] > 0.80)]

# train_features=train_features1.drop(to_drop, axis=1).copy()

### Binning target variable

In [8]:
# Binning the numeric variable to different categories
target_std=train_features['target'].std()
max_target=train_features['target'].max()+1
min_target=train_features['target'].min()-1
bins=[min_target,-3*target_std,-2*target_std,-1*target_std,target_std*1,target_std*2,target_std*3,max_target]
labels=[-4,-3,-2,0,2,3,5]
train_features['target_bin']=pd.cut(train_features['target'],bins=bins,labels=labels).astype(int)
train_features['target_bin']=train_features['target_bin'].abs()
train_features.groupby(['target_bin']).size()


target_bin
0    191093
2      8200
3       363
4      2237
5        24
dtype: int64

### KFold Splits

In [9]:
%%time
kfolds = kfold_split(5,train_features,['first_active_month', 'card_id','target','target_bin','year_nunique_y','year_nunique_x', 'outliers', 'Unnamed: 0'])

Wall time: 8.99 s


### LGBM

In [10]:
def run_lgb(train_X, train_y, val_X, val_y):
    param = {#'num_leaves': 30,
             'min_data_in_leaf': 177,
             'objective': 'regression',
             'max_depth': 14,
             'learning_rate': 0.01,
             "boosting": "gbdt",
#              "feature_fraction": 0.7,
             "bagging_freq": 1,
             "bagging_fraction": 0.7,
             "bagging_seed": 11,
             "metric": 'rmse',
             "lambda_l1": 0.1,
             "random_state": 133,
             "verbosity": -1}

    lgtrain = lgb.Dataset(train_X, label=train_y)
    lgval = lgb.Dataset(val_X, label=val_y)
    evals_result = {}
    model = lgb.train(param, lgtrain, 2000, valid_sets=[lgval], early_stopping_rounds=100, verbose_eval=100,
                      evals_result=evals_result)
    return model, evals_result

In [11]:
best_list = []
imp_list = []
for split in range(0,1):
    model, evals_result = run_lgb(kfolds[split][0], kfolds[split][2], kfolds[split][1], kfolds[split][3])
    
    best = {}
    best['best_iter'] = model.best_iteration
    best['best_score'] = model.best_score['valid_0']['rmse']
    best_list.append(best)
    feature_imp = pd.DataFrame(model.feature_importance(), kfolds[split][0].columns).sort_values(0,ascending=False).reset_index()
    feature_imp['split'] = split
    imp_list.append(feature_imp)
    
    

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's rmse: 3.69881
[200]	valid_0's rmse: 3.66561
[300]	valid_0's rmse: 3.65403
[400]	valid_0's rmse: 3.64839
[500]	valid_0's rmse: 3.64493
[600]	valid_0's rmse: 3.64352
[700]	valid_0's rmse: 3.6426
[800]	valid_0's rmse: 3.64209
[900]	valid_0's rmse: 3.64215
Early stopping, best iteration is:
[845]	valid_0's rmse: 3.6419


In [12]:
imp_list_df = pd.concat(imp_list, axis=0).groupby('index').agg('mean')
imp_list_df

,0,split
index,,
1_x,51.00,0
1_y,64.00,0
2_x,2.00,0
2_y,10.00,0
3_x,5.00,0
3_y,39.00,0
4_x,18.00,0
4_y,16.00,0
5_x,5.00,0


In [13]:
# %matplotlib inline 
# import matplotlib.pyplot as plt
# fig, ax = plt.subplots(figsize=(12,10))
# lgb.plot_importance(model, max_num_features=50, height=0.8, ax=ax)
# ax.grid(False)
# plt.title("LightGBM - Feature Importance", fontsize=15)
# plt.show()

In [23]:
train_features1 = train_features.copy()
train_features1 = train_features1.fillna(0)

In [24]:
kfolds1 = kfold_split(5,train_features1,['first_active_month', 'card_id','target','target_bin','year_nunique_y','year_nunique_x', 'outliers', 'Unnamed: 0'])

In [31]:
from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor()
from sklearn.metrics import mean_squared_error

In [41]:
rf_model = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=14,
           min_samples_leaf=177,
           n_estimators=100, n_jobs=-1,
           oob_score=True, random_state=133, verbose=-1)

In [42]:
def run_rf(train_X, train_y, val_X, val_y):

#     rftrain = lgb.Dataset(train_X, label=train_y)
#     lgval = lgb.Dataset(val_X, label=val_y)
    evals_result = {}
    model = rf_model.fit(train_X, train_y)
    y_pred = model.predict(val_X)
    rmse_score = np.sqrt(mean_squared_error(val_y, y_pred))
    return model, y_pred, rmse_score

In [43]:
best_list_rf = []
imp_list_rf = []
rmse_score = []
for split in range(0,1):
    model_rf, evals_result_rf, score = run_rf(kfolds1[split][0], kfolds1[split][2], kfolds1[split][1], kfolds1[split][3])
    
    best_rf = {}
    #best_rf['best_iter'] = model_rf.best_iteration
    rmse_score.append(score)
    #best_list_rf.append(best)
    feature_imp_rf = pd.DataFrame(model_rf.feature_importances_, kfolds1[split][0].columns).sort_values(0,ascending=False).reset_index()
    feature_imp_rf['split'] = split
    imp_list_rf.append(feature_imp_rf)

C:\Users\HP\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  8.1min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.2s finished


In [44]:
feature_imp_rf.sort_values(0, ascending=False)

,index,0,split
0,hist_month_diff_mean,0.188329,0
1,hist_category_1_sum,0.124626,0
2,new_hist_purchase_amount_var,0.081323,0
3,new_hist_month_lag_mean,0.064342,0
4,hist_month_nunique,0.064299,0
5,hist_weekofyear_nunique,0.032305,0
6,hist_authorized_flag_mean,0.022595,0
7,new_hist_purchase_amount_max,0.021212,0
8,new_hist_purchase_date_diff,0.019960,0
9,cat1_trend_coef,0.014355,0


In [45]:
rmse_score

[3.663527263070328]